In [52]:
import pandas as pd
import numpy as np

In [53]:
def rename_areas(text: str) -> str:
    if text in ['administração pública e de empresas, ciências contábeis e turismo', 'administração, ciências contábeis e turismo']:
        return 'administração pública e de empresas, ciências contábeis e turismo'
    elif text in ['arquitetura e urbanismo', 'arquitetura, urbanismo e design']:
        return 'arquitetura, urbanismo e design'
    elif text in 'filosofia/teologia:subcomissão filosofia':
        return 'filosofia'
    elif text in ['filosofia/teologia:subcomissão teologia', 'teologia']:
        return 'ciências da religião e teologia'
    elif text in 'letras / linguística':
        return 'linguística e literatura'
    elif text in 'artes / música':
        return 'artes'
    elif text in 'ciências sociais aplicadas i':
        return 'serviço social'
    else:
        return text

In [54]:
colegios_dict = {
    'ciências da vida': {
        'ciências agrárias': ['ciência de alimentos', 'ciências agrárias i', 'medicina veterinária', 'zootecnia / recursos pesqueiros'],
        'ciências biológicas': ['biodiversidade', 'ciências biológicas i', 'ciências biológicas ii', 'ciências biológicas iii'],
        'ciências da saúde': ['educação física', 'enfermagem', 'farmácia', 'medicina i', 'medicina ii', 'medicina iii', 'nutrição', 'odontologia', 'saúde coletiva']
    },
    'humanidades': {
        'ciências humanas': ['antropologia / arqueologia', 'ciência política e relações internacionais', 'ciências da religião e teologia', 'educação', 'filosofia', 'geografia', 'história', 'psicologia', 'sociologia'],
        'ciências sociais aplicadas': ['administração pública e de empresas, ciências contábeis e turismo', 'arquitetura, urbanismo e design', 'comunicação e informação', 'direito', 'economia', 'planejamento urbano e regional / demografia', 'serviço social'],
        'linguística, letras e artes': ['artes', 'linguística e literatura']
    },
    'ciências exatas, tecnológicas e multidisciplinar': {
        'ciências exatas e da terra': ['astronomia / física', 'ciência da computação', 'geociências', 'matemática / probabilidade e estatística', 'química'],
        'engenharias': ['engenharias i', 'engenharias ii', 'engenharias iii', 'engenharias iv'],
        'multidisciplinar': ['biotecnologia', 'ciências ambientais', 'ensino', 'interdisciplinar', 'materiais']
    }
}

In [55]:
for year in np.arange(2013, 2021):
    df = pd.read_csv(f'../datasets/raw/{year}.csv', sep=';', encoding ='latin')
    df = df[['AN_BASE', 'NM_PROJETO', 'DS_RESUMO', 'DS_PALAVRA_CHAVE', 
             'DS_ABSTRACT', 'DS_KEYWORD', 'NM_AREA_AVALIACAO']]
    df.columns = df.columns.str.slice(3).str.lower()
    df.rename(columns={
        'base': 'ano'
    }, inplace=True)

    df.ano = df.ano.astype('int16')

    for item in df.iloc[:, 1:].columns:
        df[item] = df[item].str.lower()
        
    df.dropna(subset='area_avaliacao', inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.area_avaliacao = df.area_avaliacao.apply(rename_areas)
        
    df['colegios'] = ''
    df['areas'] = ''
    df.rename(columns={'area_avaliacao': 'subareas'}, inplace=True)

    for colegios, areas in colegios_dict.items():
        for areas, subareas in areas.items():
            df.loc[df.subareas.isin(subareas), 'colegios'] = colegios
            df.loc[df.subareas.isin(subareas), 'areas'] = areas
        
    df.to_parquet(f'../datasets/raw/{year}.parquet', engine='pyarrow', compression='gzip')